# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,301.14,69,75,4.63,CC,1734081592
1,1,port-aux-francais,-49.3500,70.2167,278.47,81,55,15.67,TF,1734081593
2,2,grytviken,-54.2811,-36.5092,277.90,57,45,3.74,GS,1734081595
3,3,serchhip,23.3000,92.8333,300.59,20,96,1.45,IN,1734081596
4,4,malango,-2.9627,119.9001,295.28,98,100,0.17,ID,1734081597


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color= 'City'    
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
vacay1 = city_data_df[city_data_df['Humidity'] > 70]
vacay2 = vacay1[vacay1['Cloudiness'] < 50]
vacay3 = vacay2[vacay2['Wind Speed'] < 5 ]
# Drop any rows with null values
# YOUR CODE HERE
vacay_clean = vacay3.dropna()
# Display sample data
# YOUR CODE HERE
vacay_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,arraial do cabo,-22.9661,-42.0278,295.13,78,40,4.12,BR,1734081617
17,17,waitangi,-43.9535,-176.5597,288.05,96,14,1.79,NZ,1734081619
19,19,anadyr,64.7500,177.4833,271.43,86,40,3.00,RU,1734081621
37,37,balikpapan,-1.2675,116.8289,300.70,78,20,1.03,ID,1734081646
43,43,suva,-18.1416,178.4415,297.86,100,20,0.00,FJ,1734081558
...,...,...,...,...,...,...,...,...,...,...
566,566,junin de los andes,-39.9504,-71.0694,280.89,88,11,2.98,AR,1734082387
567,567,sechura,-5.5569,-80.8222,291.56,85,30,3.54,PE,1734082388
571,571,batticaloa,7.7102,81.6924,301.27,76,8,3.46,LK,1734082394
574,574,verneuil-sur-avre,48.7395,0.9273,274.74,84,31,2.81,FR,1734082397


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = vacay_clean[['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,arraial do cabo,BR,-22.9661,-42.0278,78,
17,waitangi,NZ,-43.9535,-176.5597,96,
19,anadyr,RU,64.7500,177.4833,86,
37,balikpapan,ID,-1.2675,116.8289,78,
43,suva,FJ,-18.1416,178.4415,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    "categories": "beach.beach_resort",
    "limit":1,
    "apiKey":geoapify_key    }
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lon = row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lon
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address2 = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address2["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: Vila Galé Eco Resort de Angra
waitangi - nearest hotel: Castaway Resort
anadyr - nearest hotel: No hotel found
balikpapan - nearest hotel: Pantai Melawai
suva - nearest hotel: Fiji Palms Beach Club
kodiak - nearest hotel: Jungle beach
holualoa - nearest hotel: Sands of Kahana Resort
waspan - nearest hotel: Piales
tromso - nearest hotel: Polbadet
ushuaia - nearest hotel: Tejas Verdes
new plymouth - nearest hotel: Plage
ust-tsilma - nearest hotel: Городской пляж
kingston - nearest hotel: Doctors Cave Beach
vernon - nearest hotel: Wakepilot
morden - nearest hotel: MinneTeepee Resort beach
olinda - nearest hotel: praça central de porto
mont-laurier - nearest hotel: No hotel found
waiuku - nearest hotel: Plage
leer - nearest hotel: No hotel found
grande prairie - nearest hotel: Brain Injured Goup
meadow lakes - nearest hotel: No hotel found
sarkand - nearest hotel: No hotel found
prado - nearest hotel: Enseada Beach
guerrero negro - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
16,arraial do cabo,BR,-22.9661,-42.0278,78,Vila Galé Eco Resort de Angra
17,waitangi,NZ,-43.9535,-176.5597,96,Castaway Resort
19,anadyr,RU,64.7500,177.4833,86,No hotel found
37,balikpapan,ID,-1.2675,116.8289,78,Pantai Melawai
43,suva,FJ,-18.1416,178.4415,100,Fiji Palms Beach Club
...,...,...,...,...,...,...
566,junin de los andes,AR,-39.9504,-71.0694,88,Balneario Lago Lácar
567,sechura,PE,-5.5569,-80.8222,85,Luna Nueva
571,batticaloa,LK,7.7102,81.6924,76,Half Moon Beach Resort
574,verneuil-sur-avre,FR,48.7395,0.9273,84,Base de loisirs du Pays Mêlois


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',  
    hover_cols = ['Hotel Name','City','Country'],
    color = 'City' )


# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)